# Q1
Implement a program to calculate the average occurrences of each word in a
sentence in the attached article (Youvegottofindwhatyoulove.txt).

A. Show the top 30 most frequent occurring words and their average occurrences in a sentence.

B. According to the result, what are the characteristics of these words?

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
spark = SparkSession.builder.master("yarn").appName("user").getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 03:58:29 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/30 03:58:29 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/30 03:58:29 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/30 03:58:30 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [ ]:
path0 = 'gs://yellow_tripdata_2018_10/Youvegottofindwhatyoulove.txt'
text1 = pd.read_table(path0, header = None)

In [ ]:
rdd = sc.parallelize(text1.loc[:,0].tolist())
lines = rdd.collect()
lines

['I am honored to be with you today at your commencement from one of the finest universities in the world. I never graduated from college. Truth be told, this is the closest I’ve ever gotten to a college graduation. Today I want to tell you three stories from my life. That’s it. No big deal. Just three stories.',
 'I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit. So why did I drop out?',
 'It started before I was born. My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption. She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife. Except that when I popped out they decided at the last minute that they really wanted a girl. So my parents, who were on a waiting list, got a call in the middle of the night asking: “We have an unexpected baby boy; do you 

In [ ]:
lines1 = []
for l in lines:
  s1 = l.split('.')
  for i in range(len(s1)):
    lines1.append(s1[i])

lines1

['I am honored to be with you today at your commencement from one of the finest universities in the world',
 ' I never graduated from college',
 ' Truth be told, this is the closest I’ve ever gotten to a college graduation',
 ' Today I want to tell you three stories from my life',
 ' That’s it',
 ' No big deal',
 ' Just three stories',
 '',
 'I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit',
 ' So why did I drop out?',
 'It started before I was born',
 ' My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption',
 ' She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife',
 ' Except that when I popped out they decided at the last minute that they really wanted a girl',
 ' So my parents, who were on a waiting list, got a call in the middle of the night 

In [ ]:
lines_clean = []
for i in lines1:
  if i != '':
    lines_clean.append(i)

lines_clean

['I am honored to be with you today at your commencement from one of the finest universities in the world',
 ' I never graduated from college',
 ' Truth be told, this is the closest I’ve ever gotten to a college graduation',
 ' Today I want to tell you three stories from my life',
 ' That’s it',
 ' No big deal',
 ' Just three stories',
 'I dropped out of Reed College after the first 6 months, but then stayed around as a drop-in for another 18 months or so before I really quit',
 ' So why did I drop out?',
 'It started before I was born',
 ' My biological mother was a young, unwed college graduate student, and she decided to put me up for adoption',
 ' She felt very strongly that I should be adopted by college graduates, so everything was all set for me to be adopted at birth by a lawyer and his wife',
 ' Except that when I popped out they decided at the last minute that they really wanted a girl',
 ' So my parents, who were on a waiting list, got a call in the middle of the night askin

In [2]:
# function for cleaning the data
def clean_data(data):
    data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)
    return data

In [ ]:
# Create an empty dictionary
d = dict()
  
# Loop through each line of the file
for line in lines_clean:
    # Remove the leading spaces and newline character
    line = line.strip()
  
    # Convert the characters in line to 
    # lowercase to avoid case mismatch
    line = line.lower()
  
    # Split the line into words
    words = line.split(" ")
  
    # Iterate over each word in line
    for word in words:
        # Check if the word is already in dictionary
        if word in d:
            # Increment count of word by 1
            d[word] = d[word] + 1
        else:
            # Add the word to dictionary with count 1
            d[word] = 1

In [ ]:
from collections import Counter
d1 = Counter(d)
d1.most_common()
for k, v in d1.most_common(30):
  print('%s: %i: %.2f' % (k, v, v/len(lines_clean))) 

the: 96: 0.67
i: 86: 0.60
to: 71: 0.49
and: 66: 0.46
it: 52: 0.36
was: 48: 0.33
a: 46: 0.32
of: 41: 0.28
that: 38: 0.26
in: 34: 0.24
you: 30: 0.21
my: 30: 0.21
is: 28: 0.19
had: 22: 0.15
with: 18: 0.12
out: 18: 0.12
for: 17: 0.12
so: 17: 0.12
have: 17: 0.12
your: 16: 0.11
all: 16: 0.11
as: 15: 0.10
me: 15: 0.10
on: 15: 0.10
what: 15: 0.10
but: 14: 0.10
be: 13: 0.09
from: 13: 0.09
college: 13: 0.09
life: 13: 0.09


# Q2 

In YARN cluster mode, implement a program to calculate the average amount (“Total_amount”) in credit card trip and cash trip for different numbers of passengers, which are from one to four passengers in 2018/10 NYC Yellow Taxi trip data. In NYC Taxi data, the "Passenger_count" is a driver-entered value. Explain also how you deal with the data loss issue.

In [ ]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
# yarn cluster
# spark = SparkSession.builder.master("local[*]").appName("HW").getOrCreate()
spark = SparkSession.builder.master("yarn").appName("user").getOrCreate()
sparkcontext = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 01:29:16 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/04/30 01:29:16 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/04/30 01:29:16 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/04/30 01:29:16 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [ ]:
path1 = 'gs://yellow_tripdata_2018_10/yellow_tripdata_2018-10.csv'

# data
data_1 = spark.read.csv(path1, header=True, inferSchema=True)

In [ ]:
import pyspark.sql.functions as f

data_q2 = data_1.select("total_amount", "passenger_count", "payment_type")

In [ ]:
passenger_count = [1,2,3,4]
Credit_card_agv = []
Cash_agv = []

for i in range(1,3):
  for j in range(1, 5):
    d1 = data_q2.filter((data_q2.payment_type == i) & (data_q2.passenger_count == j)).agg({'total_amount': 'avg'}).collect()[0]["avg(total_amount)"]
    if i == 1:
      Credit_card_agv.append(d1)
    else:
      Cash_agv.append(d1)

print(Credit_card_agv)
print(Cash_agv)

[18.095616294838994, 18.82315922602104, 18.397956039882864, 18.679751424181198]
[13.682682121839113, 14.72907295625041, 14.810038897804311, 15.506981138430348]


In [ ]:
# importing module
import pyspark
  
# specify column names
columns = ['passenger_count', 'Credit_card_agv', 'Cash_agv']
  
# creating a dataframe by zipping the two lists
dataframe = spark.createDataFrame(zip(passenger_count, Credit_card_agv, Cash_agv), columns)
  
# show data frame
dataframe.show()

+---------------+------------------+------------------+
|passenger_count|   Credit_card_agv|          Cash_agv|
+---------------+------------------+------------------+
|              1|18.095616294838994|13.682682121839113|
|              2| 18.82315922602104| 14.72907295625041|
|              3|18.397956039882864|14.810038897804311|
|              4|18.679751424181198|15.506981138430348|
+---------------+------------------+------------------+

